In [ ]:
import skopt
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.svm import SVC
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize

In [ ]:
# Test!!

url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
df = pd.read_csv(url, header=None)
data = df.values

x, y = data[:, :-1], data[:, -1]

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEquidistantSelection(TransformerMixin, BaseEstimator):
    def __init__(self, n_features_to_select=None):
        self.n_features_to_select = n_features_to_select
        self.selected_features_idx = []

    def get_params(self, deep=True):
        return {"n_features_to_select": self.n_features_to_select}       

    def fit(self, X, y=None):
        n_features = X.shape[1]
        self.selected_features_idx = np.linspace(0, n_features-1, self.n_features_to_select, dtype=int)
        return self

    def transform(self, X, y=None, **kwargs):
        return X[:, self.selected_features_idx]


class FeatureSelection(TransformerMixin, BaseEstimator):
    def __init__(self, n_features=None, selected_features=None):
        self.n_features = n_features
        self.selected_features = selected_features

    # def get_params(self, deep=True):
    #     params = dict()
    #     for feature in list(self.__dict__.keys()):
    #         params[feature] = getattr(self, feature)
            
    #     return params

    # def set_params(self, **parameters):
    #     for parameter, value in parameters.items():
    #         setattr(self, parameter, value)
    #     return self

    def fit(self, X, y=None):
        # n_features = X.shape[1]
        # self.selected_features_idx = np.linspace(0, n_features-1, self.n_features_to_select, dtype=int)
        return self

    def transform(self, X, y=None, **kwargs):
        if self.selected_features:
            feature_idx = format(self.selected_features, "b").zfill(n_bits)
            feature_idx = np.array(list(map(int, feature_idx)), dtype=np.bool_)
            print(feature_idx)
            return X[:, feature_idx]
        
        return X

In [ ]:
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV


pipe = Pipeline([("transform", FeatureEquidistantSelection()), ('svc', SVC())])

params = dict()
n_features = x.shape[1]
params['transform__n_features_to_select'] = (8, 34, 'uniform')
params['svc__C'] = (1e-6, 100.0, 'log-uniform')
params['svc__gamma'] = (1e-6, 100.0, 'log-uniform')
params['svc__degree'] = (1,5)
params['svc__kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the search
search = BayesSearchCV(estimator=pipe, search_spaces=params, n_jobs=-1, cv=cv)
# perform the search
search.fit(x, y)
# report the best result
print(search.best_score_)
print(search.best_params_)

In [ ]:
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Categorical

pipe = Pipeline([("transform", FeatureEquidistantSelection()), ('svc', SVC())])

params = dict()

params['transform__selected_features'] = Categorical( 'uniform',transform='onehot', name='kernel')
params['svc__C'] = (1e-6, 100.0, 'log-uniform')
params['svc__gamma'] = (1e-6, 100.0, 'log-uniform')
params['svc__degree'] = (1,5)
params['svc__kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the search
search = BayesSearchCV(estimator=pipe, search_spaces=params, n_jobs=-1, cv=cv)
# perform the search
search.fit(x, y)
# report the best result
print(search.best_score_)
print(search.best_params_)

In [ ]:
from skopt.space import Categorical
a = Categorical(['linear', 'poly', 'rbf', 'sigmoid'], transform='onehot', name='kernel')

In [ ]:
a = FeatureSelection()
a.get_params().keys()
params = {'a':1, 'b':2}
a.set_params(**params)
a.get_params(deep=True)

In [127]:
a = FeatureSelection()
params = {'n_features':34, 'selected_features':737118218}
a.set_params(**params)
print(x.shape)
a.transform(x).shape

(351, 34)
[False False False False  True False  True False  True  True  True  True
  True False  True  True  True  True  True False False False  True False
 False False False False False False  True False  True False]


(351, 15)

In [128]:
value = 737118218
# temp = {:7.2f.}format(value, "b")

n_bits = 34
temp = format(value, "b")
temp = temp.zfill(n_bits)
print(temp)


0000101011111011111000100000001010


In [ ]:
from itertools import compress, product

def combinations(items):
    return ( set(compress(items,mask)) for mask in product(*[[0,1]]*len(items)) )
    # alternative:                      ...in product([0,1], repeat=len(items)) )

list(combinations(range(0, 24)))

In [ ]:
value = 2**(34)-1

temp = format(value, "b")
print(temp)


In [ ]:
results = list(map(int, temp))
print(results)

In [ ]:
c = np.arange(0, 10).reshape(2,5)
print(c)
print(c[:, np.array([0, 1, 0, 0, 1], dtype=np.bool_)])

In [ ]:
np.array()